# Estimation of the inflection points of a sigmoid function 2018 and 2019

In [2]:
pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 7.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd

# Reading rootpainter's data and isotope data of 2018 and 2019 experiments
data_May18_raw = pd.read_csv("./Data/CLEAN_Wheat3_Full_experiment_root_length.csv")
data_June18_raw = pd.read_csv("./Data/CLEAN_Wheat4_Full_experiment_root_length.csv")
data_July18_raw = pd.read_csv("./Data/CLEAN_Wheat5_Full_experiment_root_length.csv")
isotope_data_18 = pd.read_excel("./Data/isotope_dataframe.xlsx")

data_May19_raw = pd.read_csv("./Data/RadiMax_Wheat_Root_Data_CLEAN_May19.csv")
data_June19_raw = pd.read_csv("./Data/RadiMax_Wheat_Root_Data_CLEAN_June19.csv")
data_July19_raw = pd.read_csv("./Data/RadiMax_Wheat_Root_Data_CLEAN_July19.csv")
isotope_data_19 = pd.read_excel("./Data/Breeders_15N.xlsx")
# Importing RadiMax modules for RLD data preprocessing
from RadiMaxDataPreProcessing import (
    RL_processing,
    fun_RL_computation,
    isotope_data_preprocess,
    plot_RL,
)

data_May18, data_June18, data_July18 = RL_processing(
    data_May18_raw.copy(),
    data_June18_raw.copy(),
    data_July18_raw.copy(),Square_root=False,imageArea=20,)


# Processing 2019 RL data for getting root lengths from pixels values
data_May19, data_June19, data_July19 = RL_processing(
    data_May19_raw.copy(),
    data_June19_raw.copy(),
    data_July19_raw.copy(),
    Square_root=False,imageArea=20,)

In [4]:
#Droping missing Values [np.nan, np.inf, -np.inf]
import warnings
warnings.filterwarnings("ignore")

def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

data_May18_RL=data_May18.loc[:,['tube','Root_length','soil_depth']]
data_June18_RL=data_June18.loc[:,['tube','Root_length','soil_depth']]
data_July18_RL=data_July18.loc[:,['tube','Root_length','soil_depth']]
data_May18_RL=clean_dataset(data_May18_RL)
data_June18_RL=clean_dataset(data_June18_RL)
data_July18_RL=clean_dataset(data_July18_RL)

data_May19_RL=data_May19.loc[:,['tube','Root_length','soil_depth']]
data_June19_RL=data_June19.loc[:,['tube','Root_length','soil_depth']]
data_July19_RL=data_July19.loc[:,['tube','Root_length','soil_depth']]
data_May19_RL=clean_dataset(data_May19_RL)
data_June19_RL=clean_dataset(data_June19_RL)
data_July19_RL=clean_dataset(data_July19_RL)

In [5]:
import scipy
from scipy import interpolate
import numpy as np
from numpy import arange
from pandas import read_csv
from scipy.optimize import curve_fit
from matplotlib import pyplot
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import os
def sigmoid(x, x0,b,L,c):
    return L*(scipy.special.expit((x-x0)*b,dtype = np.float128))+c  #expit(x) = 1/(1+exp(-b*(x-x0)))+c.

def sigmoid0(x, x0,b,L):
    return L*(scipy.special.expit((x-x0)*b,dtype = np.float128))  #expit(x) = 1/(1+exp(-b*(x-x0))).
    # Ensure that the sigmoid tends to infinity in the limit (because the "c" from above is now gone)


In [6]:
def FitSigmoid(x, y, zeroInfinity=True, soilMinDepth=0, soilMaxDepth=250, visual=True):
    initPars = [0.33, -50, 2, .01] # inflection, b_sig, L, c
    if zeroInfinity:
        bounds = [[0, -np.inf, 0], [1, 0, np.inf]]  # inflection, b, L
        args, cov = curve_fit(sigmoid0, x, y, p0=initPars[:-1], maxfev=1e05,ftol=1e-07, xtol=1e-07, gtol=.0001, bounds=bounds) 
        x0_sig_inflec,b_sig,L = args
        c = 0
        y_pred = sigmoid0(x, *args)
        y0_SI = sigmoid0(x0_sig_inflec, *args)
    else:
        args, cov = curve_fit(sigmoid, x, y, p0=initPars,maxfev=1000000,ftol=1e-08, xtol=1e-08,gtol=.00001) 
        x0_sig_inflec,b_sig,L,c = args    
        y_pred = sigmoid(x, *args)
        y0_SI = sigmoid(x0_sig_inflec, *args)
        
    #print('Fitted sigmoid, max L %.1f, infl %.2f, slope b %.1f, c %.1f' % (L,x0_sig_inflec,b_sig,c))

    # Estimated parameters #expit(x) = 1/(1+exp(-b*(x-x0))) : def sigmoid(x, x0,b,L,c):.
    SR2 = r2_score(y, y_pred)

    # Original soil depth scale 
    soilDepthRange = soilMaxDepth - soilMinDepth
    x0_sig_inflec_actual = round(x0_sig_inflec*soilDepthRange + soilMinDepth, 5)
    s = str(x0_sig_inflec_actual)+" cm"

    if visual:
        # Original soil depth
        xp_lav = round(x*soilDepthRange + soilMinDepth,2)   

        fig, ax = plt.subplots(figsize=(6, 3))
        ax.plot(xp_lav, y, 'o', xp_lav, y_pred,'-')
        ax.set_prop_cycle(color=['green', 'blue'])

        ax.annotate('Inflection: '+s, xy=(x0_sig_inflec_actual, y0_SI), xytext=(x0_sig_inflec_actual+4, int(y.max()+1)/2-.2),
                     arrowprops=dict(facecolor='black', shrink=0.05))

        ax.vlines(x=x0_sig_inflec_actual, ymin=-0.1, ymax=int(y.max()+1),color='orange')
        #plt.title('August 2020 : row no. '+ str(tube_no.astype(int)))
        ax.legend(['Sqrt_pRLD',r'Sigmoid: $R^{2}=$'+str(np.round(SR2,2))])

        ax.set_xlabel('Soil depth [cm]')
        ax.set_ylabel('Sqrt_pRLD per image (sqrt($cm^{-1}$))')

    return x0_sig_inflec_actual, L

# Without Noise

In [ ]:
Month=data_June18_RL # 2018 June 
inflection=np.array([])
dTube=np.array([])

for i in set(Month.tube): #use for all tubes in June
#for i in range(2,20): # for first few tubes
    #print('Tube=%d'%i)
    temp1 = Month[Month.tube==i].sort_values(by='soil_depth').reset_index(drop=True)
    x = temp1['soil_depth']
    y = temp1['Root_length']
    #std=0.1*y.std()
    x = (x- x.values[0])/(x.values[-1] - x.values[0])   # normalize   
    #y = y + np.random.normal(0, std, size=len(y))
    #y[y<0]=0
    x0_sig_inflec_actual,_=FitSigmoid(x, np.sqrt(y), zeroInfinity=True, soilMinDepth=temp1['soil_depth'].values[0], soilMaxDepth=temp1['soil_depth'].values[-1],visual=False)
   # print('SI=%0.0f'%x0_sig_inflec_actual)
    
    inflection=np.append(inflection,x0_sig_inflec_actual)
    dTube=np.append(dTube,i)

df['tube']=dTube
df['SI_June']=inflection

# Gaussian Noise

In [8]:
df=pd.DataFrame()

In [13]:
Month=data_June18_RL
inflection=np.array([])
dTube=np.array([])

for i in set(Month.tube): #use for all tubes in June
#for i in range(2,20): # for first few tubes
    #print('Tube=%d'%i)
    temp1 = Month[Month.tube==i].sort_values(by='soil_depth').reset_index(drop=True)
    x = temp1['soil_depth']
    y = temp1['Root_length']
    std=0.1*y.std() # Noise level 10 % of std 
    x = (x- x.values[0])/(x.values[-1] - x.values[0])   # normalize   
    y = y + np.random.normal(0, std, size=len(y))
    y[y<0]=0
    x0_sig_inflec_actual,_=FitSigmoid(x, np.sqrt(y), zeroInfinity=True, soilMinDepth=temp1['soil_depth'].values[0], soilMaxDepth=temp1['soil_depth'].values[-1],visual=False)
   # print('SI=%0.0f'%x0_sig_inflec_actual)
    
    inflection=np.append(inflection,x0_sig_inflec_actual)
    dTube=np.append(dTube,i)

df['tube']=dTube
df['SI_June18_10%_Noise']=inflection

In [14]:
df

,tube,SI_June18,SI_June18_1%_Noise,SI_June18_10%_Noise
0,1.0,120.48085,120.48085,120.48085
1,2.0,149.84505,149.85460,150.21205
2,3.0,147.65474,147.74245,147.85419
3,4.0,152.15179,151.49828,150.32595
4,5.0,155.41169,155.76525,156.38577
...,...,...,...,...
289,296.0,178.72504,179.20648,180.09906
290,297.0,162.29493,159.82207,158.84775
291,298.0,165.71788,163.60632,158.06441
292,299.0,181.96302,181.94661,182.65430


In [55]:
df1.to_csv("SI_July-18.csv")

In [15]:
df.to_csv("SI_June_18_new.csv")